In [2]:
import tweepy
from tweepy import OAuthHandler
import json
import datetime as dt
import time
import os
import sys
import numpy as np
import pandas as pd
import json
import csv

In [3]:
import re
import string
from geopy.geocoders import Nominatim

In [4]:
def create_csv(filename,file_list):
    geo_count = 0 
    without_geo = 0
    total_count = 0
    others = 0
    csvfile = open(filename, "w")
    f = csv.writer(csvfile)
 
    f.writerow(['tweet', # relabeled: the API calls this 'text' 
                'country_code', 
                'geo_location', # relabeled: the API calls this 'full_name' 
                'bounding_box',
                'latitude', 
                'longitude',
                'User_name', 
                'favourites_count', 
                'followers_count', 
                'statuses_count', 
                'friends_count', 
                'listed_count', 
                'user_described_location', # relabeled: the API calls this 'location' 
                'created_at', 
                'utc_offset']) 
    
    for file_ in file_list: 
        with open(file_, "r") as r: 
            for line in r: 
                try: 
                    tweet = json.loads(line) 
                except: 
                    continue 
                if tweet and tweet['place'] != None and tweet['geo'] != None: 
                    f.writerow([tweet['full_text'], 
                                tweet['place']['country_code'], 
                                tweet['place']['full_name'], 
                                tweet['place']['bounding_box']['coordinates'], 
                                tweet['geo']['coordinates'][0],
                                tweet['geo']['coordinates'][1],
                                tweet['user']['screen_name'], 
                                tweet['user']['favourites_count'], 
                                tweet['user']['followers_count'], 
                                tweet['user']['statuses_count'], 
                                tweet['user']['friends_count'], 
                                tweet['user']['listed_count'], 
                                tweet['user']['location'], 
                                tweet['created_at'], 
                                tweet['user']['utc_offset']])
                    geo_count +=1
                elif tweet and tweet['place'] != None and tweet['geo'] == None:
                    f.writerow([tweet['full_text'], 
                                tweet['place']['country_code'], 
                                tweet['place']['full_name'], 
                                None, 
                                None,
                                None,
                                tweet['user']['screen_name'], 
                                tweet['user']['favourites_count'], 
                                tweet['user']['followers_count'], 
                                tweet['user']['statuses_count'], 
                                tweet['user']['friends_count'], 
                                tweet['user']['listed_count'], 
                                tweet['user']['location'], 
                                tweet['created_at'], 
                                tweet['user']['utc_offset']])
                    without_geo +=1
                elif tweet and tweet['place'] == None and tweet['geo'] == None:
                    f.writerow([tweet['full_text'], 
                                None, 
                                None, 
                                None, 
                                None,
                                None,
                                tweet['user']['screen_name'], 
                                tweet['user']['favourites_count'], 
                                tweet['user']['followers_count'], 
                                tweet['user']['statuses_count'], 
                                tweet['user']['friends_count'], 
                                tweet['user']['listed_count'], 
                                tweet['user']['location'], 
                                tweet['created_at'], 
                                tweet['user']['utc_offset']])
                    others += 1
                total_count+=1
            
    print('Total Tweets ==>{}'.format(total_count))
    print('Tweets with geo_tags ==>{}'.format(geo_count))
    print('Tweets without geo_tags==>{}'.format(without_geo))
    print('Tweets without geo_tags and place info==>{}'.format(others))
    

#### Prepare CSV File for Delhi Tweets

In [8]:
file_list = ['Delhi_2019-03-11.json','Delhi_2019-03-12.json','Delhi_2019-03-13.json','Delhi_2019-03-18.json','Delhi_2019-03-19.json','Delhi_2019-03-20.json']
filename = 'delhi_march_analysis.csv'
create_csv(filename,file_list)


Total Tweets ==>965423
Tweets with geo_tags ==>685
Tweets without geo_tags==>20378
Tweets without geo_tags and place info==>944360


In [9]:
file_list = ['Hyderabad_2019-03-11.json','Hyderabad_2019-03-12.json','Hyderabad_2019-03-13.json','Hyderabad_2019-03-18.json','Hyderabad_2019-03-19.json','Hyderabad_2019-03-20.json']
filename = 'hyderabad_march_analysis.csv'
create_csv(filename,file_list)

Total Tweets ==>485216
Tweets with geo_tags ==>820
Tweets without geo_tags==>12697
Tweets without geo_tags and place info==>471699


In [10]:
file_list = ['Mumbai_2019-03-11.json','Mumbai_2019-03-12.json','Mumbai_2019-03-13.json','Mumbai_2019-03-18.json','Mumbai_2019-03-19.json','Mumbai_2019-03-20.json']
filename = 'mumbai_march_analysis.csv'
create_csv(filename,file_list)

Total Tweets ==>924893
Tweets with geo_tags ==>2192
Tweets without geo_tags==>25608
Tweets without geo_tags and place info==>897093


In [11]:
file_list = ['Bangalore_2019-03-11.json','Bangalore_2019-03-12.json','Bangalore_2019-03-13.json','Bangalore_2019-03-18.json','Bangalore_2019-03-19.json','Bangalore_2019-03-20.json']
filename = 'bangalore_march_analysis.csv'
create_csv(filename,file_list)

Total Tweets ==>635684
Tweets with geo_tags ==>1625
Tweets without geo_tags==>24098
Tweets without geo_tags and place info==>609961


In [5]:
df_delhi = pd.read_csv('delhi_march_analysis.csv', encoding='utf-8')
df_mumbai = pd.read_csv('mumbai_march_analysis.csv', encoding='utf-8')
df_banglore = pd.read_csv('bangalore_march_analysis.csv', encoding='utf-8')
df_hyderabad = pd.read_csv('hyderabad_march_analysis.csv', encoding='utf-8')

In [6]:
print('Total Tweets collected from delhi==>{}'.format(len(df_delhi)))
print('Total Tweets collected from mumbai==>{}'.format(len(df_mumbai)))
print('Total Tweets collected from bangalore==>{}'.format(len(df_banglore)))
print('Total Tweets collected from hyderabad==>{}'.format(len(df_hyderabad)))

Total Tweets collected from delhi==>965423
Total Tweets collected from mumbai==>924893
Total Tweets collected from bangalore==>635684
Total Tweets collected from hyderabad==>485216


#### Removing Reweets

In [7]:
#df_delhi = df_delhi[~df_delhi.tweet.str.startswith('RT')]
df_mumbai = df_mumbai[~df_mumbai.tweet.str.startswith('RT')]
df_banglore = df_banglore[~df_banglore.tweet.str.startswith('RT')]
df_hyderabad = df_hyderabad[~df_hyderabad.tweet.str.startswith('RT')]

#### After Removing Retweets Remaining Tweets in each dataframe:

In [8]:
#print('Remaining Tweets collected from delhi==>{}'.format(len(df_delhi)))
print('Remaining Tweets collected from mumbai==>{}'.format(len(df_mumbai)))
print('Remaining Tweets collected from bangalore==>{}'.format(len(df_banglore)))
print('Remaining Tweets collected from hyderabad==>{}'.format(len(df_hyderabad)))

Remaining Tweets collected from mumbai==>372410
Remaining Tweets collected from bangalore==>321250
Remaining Tweets collected from hyderabad==>217557


#### PreProcess Tweets - remove @usermentions, URL Links,emoticons

In [9]:
# Clean tweets
# remove duplicate tweets - already done
# remove @usermentions
# remove URL Links
# remove emoticons
from bs4 import BeautifulSoup
from nltk.tokenize import WordPunctTokenizer
tok = WordPunctTokenizer()

In [10]:
pat1 = r'@[A-Za-z0-9]+'
pat2 = r'https?://[A-Za-z0-9./]+'
combined_pat = r'|'.join((pat1, pat2))

def tweet_cleaner(row):
    stripped = re.sub(combined_pat, '', row['tweet'])
    soup = BeautifulSoup(stripped, 'lxml')
    souped = soup.get_text()
    try:
        clean = souped.decode("utf-8-sig").replace(u"\ufffd", "?")
    except:
        clean = souped
    letters_only = re.sub("[^a-zA-Z]", " ", clean)
    lower_case = letters_only.lower()
    # During the letters_only process two lines above, it has created unnecessay white spaces,
    # I will tokenize and join together to remove unneccessary white spaces
    words = tok.tokenize(lower_case)
    clean_tweet = (" ".join(words)).strip()
    if len(clean_tweet) > 0:
        return clean_tweet
    else:
        return None

In [11]:
#df_delhi['Cleaned_tweets'] = df_delhi.apply (lambda row: tweet_cleaner(row), axis=1)
df_mumbai['Cleaned_tweets'] = df_mumbai.apply (lambda row: tweet_cleaner(row), axis=1)
df_hyderabad['Cleaned_tweets'] = df_hyderabad.apply (lambda row: tweet_cleaner(row), axis=1)
df_banglore['Cleaned_tweets'] = df_banglore.apply (lambda row: tweet_cleaner(row), axis=1)

/home/nishant/anaconda3/lib/python3.7/site-packages/bs4/__init__.py:273: UserWarning: "b'.'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)
/home/nishant/anaconda3/lib/python3.7/site-packages/bs4/__init__.py:336: UserWarning: "http:/s/apib.nic.einard/?PRID=e1569106#.XJDubLQHcMU.twitter" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup


In [11]:
df_mumbai.head(10)

,tweet,country_code,geo_location,bounding_box,latitude,longitude,User_name,favourites_count,followers_count,statuses_count,friends_count,listed_count,user_described_location,created_at,utc_offset,Cleaned_tweets
8,@MosesSamPaul @free_thinker @AltNews Most of t...,NaN,NaN,NaN,NaN,NaN,vnay_85,15730,121,35819,431,0,"Navi Mumbai, India",Mon Mar 11 23:59:50 +0000 2019,NaN,thinker most of their funds come from terroris...
11,"""تیرے تو وَصف عیب تناہی سے ہیں بَری ::\nحیراں ...",NaN,NaN,NaN,NaN,NaN,SyedMuhammadQu3,1999,90,1392,39,0,"मुंबई, भारत",Mon Mar 11 23:59:44 +0000 2019,NaN,None
14,@mahesh10816 Tell me which party has content. ...,NaN,NaN,NaN,NaN,NaN,SopaliTeena,6,140,555,1585,0,"Mumbai, India",Mon Mar 11 23:59:35 +0000 2019,NaN,tell me which party has content every one set ...
17,@jacksfilms #YIAYdroid pls buy our fuirsuit,NaN,NaN,NaN,NaN,NaN,Esh66813127,288,2,32,57,0,"Mumbai, India",Mon Mar 11 23:59:25 +0000 2019,NaN,yiaydroid pls buy our fuirsuit
18,When the entire team is having panic attacks a...,NaN,NaN,NaN,NaN,NaN,AniketSMK,32358,855,7266,3531,7,"Mumbai, India",Mon Mar 11 23:59:25 +0000 2019,NaN,when the entire team is having panic attacks a...
19,#तुम्ही_यशाच्या_रेषे_ला_टच_करताच_लोक_तुमचे_पाय...,NaN,NaN,NaN,NaN,NaN,gosaviakash2,48100,224,35159,1267,1,"Mumbai, India",Mon Mar 11 23:59:24 +0000 2019,NaN,None
24,@YogiAdityanath_ @RahulGandhi मामा,NaN,NaN,NaN,NaN,NaN,anilrajbhar2,38014,345,5882,483,0,mumbai,Mon Mar 11 23:59:14 +0000 2019,NaN,None
26,@Sulci_Gyri @timesofindia The fact that our na...,NaN,NaN,NaN,NaN,NaN,ShindeAvishkar,121,25,104,60,1,"Thane, Maharashtra",Mon Mar 11 23:59:13 +0000 2019,NaN,gyri the fact that our nation still mourns the...
27,@RealBrianWatt Is that EURUSD?,NaN,NaN,NaN,NaN,NaN,HybridInvest,5570,7827,12523,164,136,"Bombay, India",Mon Mar 11 23:59:11 +0000 2019,NaN,is that eurusd
30,@iamAnkita_y @RahulGandhi मामा दिग्विजय के लिए...,IN,"Mumbai, India",NaN,NaN,NaN,jbhimrajka,2054,128,4810,507,0,"Mumbai, India",Mon Mar 11 23:59:09 +0000 2019,NaN,y


#### Removing Rows where cleaned tweets are empty

In [12]:
df_mumbai = df_mumbai[df_mumbai.Cleaned_tweets.notnull()]
df_hyderabad = df_hyderabad[df_hyderabad.Cleaned_tweets.notnull()]
df_bangalore = df_banglore[df_banglore.Cleaned_tweets.notnull()]

In [13]:
print('After Cleaning Tweets remaining from mumbai==>{}'.format(len(df_mumbai)))
print('After Cleaning Tweets remaining from bangalore==>{}'.format(len(df_bangalore)))
print('After Cleaning Tweets remaining from hyderabad==>{}'.format(len(df_hyderabad)))

After Cleaning Tweets remaining from mumbai==>316634
After Cleaning Tweets remaining from bangalore==>279824
After Cleaning Tweets remaining from hyderabad==>187541


#### To Analyze if Location from Geo-Coordinates and places from tweets json object are similar, filter the tweets containing geo-coordinate and place info

In [14]:
df_geo_mumbai = df_mumbai.loc[df_mumbai['longitude'].notnull()]
df_geo_hyderabad = df_hyderabad.loc[df_hyderabad['longitude'].notnull()]
df_geo_bangalore = df_bangalore.loc[df_bangalore['longitude'].notnull()]

In [15]:
print('Tweets containing geo-info and place info from mumbai location==>{}'.format(len(df_geo_mumbai)))
print('Tweets containing geo-info and place info from bangalore location==>{}'.format(len(df_geo_bangalore)))
print('Tweets containing geo-info and place info from hyderabad location==>{}'.format(len(df_geo_hyderabad)))

Tweets containing geo-info and place info from mumbai location==>2170
Tweets containing geo-info and place info from bangalore location==>1617
Tweets containing geo-info and place info from hyderabad location==>809


In [16]:
df_geo_mumbai = df_geo_mumbai[['geo_location','latitude','longitude','user_described_location']]
df_geo_hyderabad = df_geo_hyderabad[['geo_location','latitude','longitude','user_described_location']]
df_geo_bangalore = df_geo_bangalore[['geo_location','latitude','longitude','user_described_location']]

In [17]:
#unique places 
df_geo_mumbai['geo_location'].unique()

array(['Mumbai, India', 'Thane, India', 'Navi Mumbai, India',
       'Panvel, India', 'Ulhasnagar, India', 'Haveli, India',
       'Maharashtra, India', 'Kalyan Dombivali, India', 'Chakan, India',
       'Lonavala, India', 'Alibagh, India', 'Bhiwandi, India',
       'Murbad, India', 'PF office', 'Talegaon Dabhade, India',
       'Manisha Nagar', 'Vishnu Shivam Tower', 'Karjat, India',
       'Sarvoli, India', 'Jhula Maidan',
       'Asiatic Society Library (Town Hall)', 'Khalapur, India',
       'Uran, India', 'Vashivali Industrial Area, India',
       'Aamby Valley, India', 'Carribean tower', 'Andheri West, Mumbai',
       'igatpuri', 'Versova, Mumbai',
       'Jeevan Rachna Co-Operative Housing Society Ltd', 'Cool Camp',
       'Sewri Railway Station', 'India', 'Majholi, India',
       'Pest, Hungary'], dtype=object)

In [18]:
df_geo_hyderabad['geo_location'].unique()

array(['Rajendra Nagar, India', 'Hyderabad, India', 'Hayathnagar, India',
       'India', 'Drasrao Nagar, Hayathnagar', 'Medchal, India',
       'Andhra Pradesh, India', 'Sangareddi, India',
       'Ibrahimpatan, India', 'Kukatpally', 'Vijayawada, India',
       'Ghatkesar, India', 'Farooq Nagar, India', 'Narsapur, India',
       'Bhongir, India', 'Ecil Cricket Ground',
       'karmanghat Hanuman temple', 'Nalgonda, India',
       'Yadagirigutta, India', 'green park colony Karmanghat',
       'Mujtaba Jewellers', 'malakpet', 'ECIL', 'City Light Hotel',
       'Kondapur', 'Academic Heights Public School', 'ECIL, Hayathnagar',
       'Sainathpuram', 'Kakinada, India', 'DAE colony'], dtype=object)

In [19]:
df_geo_bangalore['geo_location'].unique()

array(['Bengaluru, India', 'Karnataka, India',
       'Banashankari, Bengaluru South', 'Bengaluru South, India',
       'Anekal Bangaluru, India', 'Hosakote, India', 'Malur, India',
       'Hosur, India', 'Hosur SIPCOT Phase 2, India', 'Tamil Nadu, India',
       'Vijayawada, India', 'Devanahalli, India', 'Tumkur, India',
       'Mookandapalli, India', 'BSNL Office', 'Nelamangala, India',
       'B Narayanapura Bus Stop', 'RMZ Infinity', 'The Lalit Ashok',
       'Nandi, India', 'Dod Ballapur, India', 'Hassan, India',
       'Kolar, India', 'Vaniyambadi, India', 'Ambur, India',
       'Someshwara Nagar, Bengaluru', 'K.R.Market',
       'Manyata Embassy Business Park', 'Gubbi, India',
       'Coffee Mechanics', 'Costa Coffee', 'Indian Public School',
       'Jaipur, India', 'Annex, OR', 'Shell Select', 'LRDE',
       'Ramanagaram, India', 'Swami Vivekananda Road Metro Station',
       'BTM Layout, Bengaluru South', 'Mulbagal, India'], dtype=object)

#### Getting Address from Lat/Long using OpenStreetMap APIs

In [21]:
def find_addr(row):
    #print(row['latitude'])
    time.sleep(1)
    addr =  str(row['latitude']) + ',' + str(row['longitude'])
    #print(addr)
    try:
        geolocator = Nominatim(user_agent="College Project")
        location = geolocator.reverse(addr)
         #print(results[0])
        if len(location) > 0:
            #print(location.address)
            return location.address
        else:
            return None
    except:
        print("Exception occured")
        return None

In [22]:
df_geo_mumbai['Add_From_latlong'] = df_geo_mumbai.apply (lambda row: find_addr(row), axis=1)

Exception occured
Exception occured
Exception occured
Exception occured
Exception occured
Exception occured
Exception occured
Exception occured
Exception occured
Exception occured
Exception occured
Exception occured
Exception occured
Exception occured
Exception occured
Exception occured
Exception occured
Exception occured
Exception occured
Exception occured
Exception occured
Exception occured
Exception occured
Exception occured
Exception occured
Exception occured
Exception occured
Exception occured
Exception occured
Exception occured
Exception occured
Exception occured
Exception occured
Exception occured
Exception occured
Exception occured
Exception occured
Exception occured
Exception occured
Exception occured
Exception occured
Exception occured
Exception occured
Exception occured
Exception occured
Exception occured


In [23]:
df_geo_hyderabad['Add_From_latlong'] = df_geo_hyderabad.apply (lambda row: find_addr(row), axis=1)

Exception occured
Exception occured
Exception occured
Exception occured
Exception occured
Exception occured
Exception occured
Exception occured
Exception occured
Exception occured
Exception occured
Exception occured
Exception occured
Exception occured
Exception occured


In [24]:
df_geo_bangalore['Add_From_latlong'] = df_geo_bangalore.apply (lambda row: find_addr(row), axis=1)

Exception occured
Exception occured
Exception occured
Exception occured
Exception occured
Exception occured
Exception occured
Exception occured
Exception occured
Exception occured
Exception occured
Exception occured
Exception occured
Exception occured
Exception occured
Exception occured
Exception occured
Exception occured
Exception occured
Exception occured
Exception occured
Exception occured
Exception occured
Exception occured
Exception occured
Exception occured


In [28]:
df_geo_bangalore.head()

,geo_location,latitude,longitude,user_described_location,Add_From_latlong
348,"Bengaluru, India",12.971600,77.594600,NaN,"St Joseph's Indian High School, Vittal Mallya ..."
637,"Bengaluru, India",12.993184,77.531419,India,"Shakthi Ganapathi Nagar Ward, West Zone, Benga..."
951,"Bengaluru, India",12.993184,77.531419,India,"Shakthi Ganapathi Nagar Ward, West Zone, Benga..."
1289,"Bengaluru, India",12.970214,77.560289,"Indore, Madhya Pradesh","Binnipete Ward, West Zone, Bengaluru, Bangalor..."
1290,"Bengaluru, India",12.970214,77.560289,"Indore, Madhya Pradesh","Binnipete Ward, West Zone, Bengaluru, Bangalor..."


In [29]:
df_geo_hyderabad.head()

,geo_location,latitude,longitude,user_described_location,Add_From_latlong
145,"Rajendra Nagar, India",17.416000,78.439003,thunder bay,"Gauri Shankar Nagar, Ward 92 Venkateshwara Col..."
684,"Hyderabad, India",17.403687,78.509834,NaN,"Padma Colony, Ward 85 Adikmet, Greater Hyderab..."
828,"Rajendra Nagar, India",17.424787,78.429585,Hyderabad,"Park Hyatt, Road No. 3, BN Reddy Colony, Ward ..."
1517,"Hyderabad, India",17.402259,78.509370,India,"Padma Colony, Ward 85 Adikmet, Greater Hyderab..."
2302,"Rajendra Nagar, India",17.419054,78.443423,Hyderabad,"Banjara Hills Road Number 7, Vimal Nagar, Ward..."


In [30]:
df_geo_mumbai.head()

,geo_location,latitude,longitude,user_described_location,Add_From_latlong
298,"Mumbai, India",19.062100,72.876400,"Mumbai, Maharashtra","Naware Baug Jagruti Nagar, L Ward, Mumbai, Mum..."
2327,"Thane, India",19.254260,72.973509,India,"Waghbil road, Brahmand Nagar, Thane, Maharasht..."
2972,"Mumbai, India",19.034818,72.843733,"Sadly, Home","Pandit Satawlekar Marg, Telephone Colony, G/N ..."
3786,"Mumbai, India",19.006140,72.836650,Toronto | Mumbai,"Shindewadi, F/S Ward, Zone 2, Mumbai, Mumbai C..."
3860,"Mumbai, India",19.107720,72.831828,Heaven Bound,"NS Road No 8, Hastiraj Plot, K/W Ward, Zone 3,..."


In [31]:
df_geo_mumbai = df_geo_mumbai.loc[df_geo_mumbai['Add_From_latlong'].notnull()]
df_geo_hyderabad = df_geo_hyderabad.loc[df_geo_hyderabad['Add_From_latlong'].notnull()]
df_geo_bangalore = df_geo_bangalore.loc[df_geo_bangalore['Add_From_latlong'].notnull()]

#### After Discarding coordinates which got exception during address retrival

In [32]:
print('mumbai location==>{}'.format(len(df_geo_mumbai)))
print('bangalore location==>{}'.format(len(df_geo_bangalore)))
print('hyderabad location==>{}'.format(len(df_geo_hyderabad)))

mumbai location==>2124
bangalore location==>1591
hyderabad location==>794


##### For future use converting df with address to csv

In [35]:
df_geo_hyderabad.to_csv('geo_hyderabad.csv', encoding='utf-8', index=False)
df_geo_mumbai.to_csv('geo_mumbai.csv', encoding='utf-8', index=False)
df_geo_bangalore.to_csv('geo_bangalore.csv', encoding='utf-8', index=False)

In [55]:
def match_place(row):
    place = str(row['geo_location']).lower()
    place_list = [x.strip() for x in place.split(',')]
    addrs = str(row['Add_From_latlong']).lower()
    matched = []
    for p in place_list:
        if p in addrs:
            matched.append(p)
    if len(matched) > 0:
        return ",".join(matched )
    else:
        return None    

In [56]:
df_geo_mumbai['Matched_Addr'] = df_geo_mumbai.apply (lambda row: match_place(row), axis=1)
df_geo_hyderabad['Matched_Addr'] = df_geo_hyderabad.apply (lambda row: match_place(row), axis=1)
df_geo_bangalore['Matched_Addr'] = df_geo_bangalore.apply (lambda row: match_place(row), axis=1)

/home/nishant/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/home/nishant/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/nishant/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the do

In [60]:
df_geo_mumbai['Matched_Addr'].value_counts()

mumbai,india           1671
india                   192
navi mumbai,india       100
thane,india              92
maharashtra,india        18
chakan,india             11
panvel,india             10
lonavala,india            4
uran,india                3
ulhasnagar,india          3
murbad,india              2
mumbai                    2
karjat,india              2
bhiwandi,india            1
aamby valley,india        1
vishnu shivam tower       1
Name: Matched_Addr, dtype: int64

In [61]:
df_geo_hyderabad['Matched_Addr'].value_counts()

hyderabad,india        385
india                  263
medchal,india           21
nalgonda,india           8
ghatkesar,india          2
bhongir,india            2
dae colony               2
yadagirigutta,india      2
malakpet                 1
narsapur,india           1
Name: Matched_Addr, dtype: int64

In [62]:
df_geo_bangalore['Matched_Addr'].value_counts()

bengaluru,india      837
india                617
hosakote,india        40
karnataka,india       35
tamil nadu,india       9
nelamangala,india      9
hosur,india            6
vaniyambadi,india      5
hassan,india           4
bengaluru              4
devanahalli,india      3
nandi,india            2
tumkur,india           2
gubbi,india            1
btm layout             1
banashankari           1
malur,india            1
kolar,india            1
or                     1
rmz infinity           1
ambur,india            1
Name: Matched_Addr, dtype: int64

### Get Location from tweet text

In [65]:
import sys
def fetch_places(filename):
    input_buffer = []
    fh = open(filename)
    for line in fh:
        place = str(line.strip()).lower()
        input_buffer.append(place)
    return input_buffer

In [66]:
hyderabad_places = fetch_places('hyderabad_places.txt')
mumbai_places = fetch_places('mumbai_places.txt')
bangalore_places = fetch_places('bangalore_places.txt')
delhi_places = fetch_places('delhi_places.txt')

In [67]:
def find_location(row,place_list):
    tweet = row['Cleaned_tweets']
    words = tweet.strip().split()
    location = []
    for w in words:
        if w in place_list:
            location.append(w)
    if len(location) > 0:
        return ",".join(location)
    else:
        return None

In [70]:
df_mumbai_tweets = df_mumbai[['Cleaned_tweets']]
df_hyderabad_tweets = df_hyderabad[['Cleaned_tweets']]
df_bangalore_tweets = df_bangalore[['Cleaned_tweets']]

In [71]:
df_mumbai_tweets['Location'] = df_mumbai_tweets.apply (lambda row: find_location(row,mumbai_places), axis=1)
df_hyderabad_tweets['Location'] = df_hyderabad_tweets.apply (lambda row: find_location(row,hyderabad_places), axis=1)
df_bangalore_tweets['Location'] = df_bangalore_tweets.apply (lambda row: find_location(row,bangalore_places), axis=1)

/home/nishant/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/home/nishant/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/nishant/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the do

In [72]:
df_bangalore_tweets['Location'].value_counts()

whitefield                       175
jayanagar                         77
yelahanka                         50
hosur                             40
srinagar                          37
marathahalli                      36
majestic                          34
rajajinagar                       33
sarjapur                          31
vijayanagar                       30
madiwala                          21
varthur                           19
hoskote                           17
kundalahalli                      17
nelamangala                       14
mahadevapura                      14
thippasandra                      11
shivajinagar                      11
yeshwanthpur                      11
kengeri                           11
vidyaranyapura                    10
malleswaram                        9
jayanagar,jayanagar                8
kadugodi                           7
peenya                             7
hosur,hosur                        6
mathikere                          6
k

In [73]:
df_hyderabad_tweets['Location'].value_counts()

ameerpet                                             187
gachibowli                                            77
charminar                                             75
madhapur                                              68
bahadurpura                                           67
kukatpally                                            63
khairatabad                                           60
paradise                                              39
kondapur                                              30
abids                                                 30
nampally                                              28
golconda                                              24
shamshabad                                            24
begumpet                                              23
malakpet                                              23
miyapur                                               23
nagole                                                22
amberpet                       

In [74]:
df_mumbai_tweets['Location'].value_counts()

bandra                                         254
andheri                                        195
juhu                                           107
kalyan                                          98
borivali                                        76
worli                                           57
malad                                           57
chembur                                         56
kandivali                                       54
panvel                                          50
ghatkopar                                       49
dadar                                           47
kharghar                                        42
fort                                            38
khar                                            38
bhiwandi                                        34
santacruz                                       33
parel                                           26
lokhandwala                                     26
churchgate                     

In [90]:
df_mumbai_tweets[df_mumbai_tweets.Cleaned_tweets.str.contains('andheri')].values

array([['officespace available in kanakia wallstreet chakala andherieast forrent mumbai commercial property',
        'andheri'],
       ['bhk sqft andheri west apartment flat forsale andheriwest mumbai residential property',
        'andheri'],
       ['officespace forrent in andherieast near metro station mumbai commercial property',
        'andheri'],
       ['bom gadkari yes you are right the vehical shown in the pic is force trax and a private jeep carring school children with proper valid documents who will held responsible if any thing goes wrong jaagoo police jaagoo andherirto',
        'andheri'],
       ['my startled sleepy ass on train when pa announces agla station andheri kripya qatar mein utre',
        'andheri'],
       ['gyancho rd march cat cafe andheri', 'andheri'],
       ['andheri east on sahar road nr pillar no towards airport',
        'andheri'],
       ['a very common sight at andheri station', 'andheri'],
       ['poorest internet in andheri east past month u